### Import packages

In [1]:
import os
import sys
import time
from datetime import datetime

import GPUtil
import psutil

#######################
# run after two days
# time.sleep(172800)

#######################
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
sys.path.append("../")

In [2]:
def gpu_free(max_gb):
    gpu_id = GPUtil.getFirstAvailable(
        order="memory"
    )  # get the fist gpu with the lowest load
    GPU = GPUtil.getGPUs()[gpu_id[0]]
    GPU_load = GPU.load * 100
    GPU_memoryUtil = GPU.memoryUtil / 2.0 ** 10
    GPU_memoryTotal = GPU.memoryTotal / 2.0 ** 10
    GPU_memoryUsed = GPU.memoryUsed / 2.0 ** 10
    GPU_memoryFree = GPU.memoryFree / 2.0 ** 10
    print(
        "-- total_GPU_memory: %.3fGB;init_GPU_memoryFree:%.3fGB init_GPU_load:%.3f%% GPU_memoryUtil:%d%% GPU_memoryUsed:%.3fGB"
        % (GPU_memoryTotal, GPU_memoryFree, GPU_load, GPU_memoryUtil, GPU_memoryUsed)
    )
    if GPU_memoryFree > max_gb:
        return True
    return False


def memery_free(max_gb):
    available_memory = psutil.virtual_memory().free / 2.0 ** 30
    if available_memory > max_gb:
        return True
    return False

In [ ]:
for item_fea_type in [
    "random",
    "cate",
    "cate_word2vec",
    "cate_bert",
    "cate_one_hot",
    "random_word2vec",
    "random_bert",
    "random_one_hot",
    "random_bert_word2vec_one_hot",
    "random_cate_word2vec",
    "random_cate_bert",
    "random_cate_one_hot",
    "random_cate_bert_word2vec_one_hot",
]:
    while True:
        if gpu_free(4) and memery_free(10):
            os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
            gpu_id = GPUtil.getAvailable(order="memory", limit=4)[
                0
            ]  # get the fist gpu with the lowest load
            print("GPU memery and main memery availale, start a job")
            date_time = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
            command = f"CUDA_VISIBLE_DEVICES=0,1,2,3; /home/zm324/anaconda3/envs/beta_rec/bin/python run_tvbr.py --item_fea_type {item_fea_type} --device cuda:{gpu_id} >> ./logs/{date_time}_{item_fea_type}.log &"
            os.system(command)
            time.sleep(120)
            break
        else:
            print("GPU not availale, sleep for 10 min")
            time.sleep(600)
            continue

-- total_GPU_memory: 10.761GB;init_GPU_memoryFree:10.760GB init_GPU_load:0.000% GPU_memoryUtil:0% GPU_memoryUsed:0.001GB
GPU memery and main memery availale, start a job
-- total_GPU_memory: 10.761GB;init_GPU_memoryFree:10.757GB init_GPU_load:0.000% GPU_memoryUtil:0% GPU_memoryUsed:0.004GB
GPU memery and main memery availale, start a job
